In [7]:
import torch
import torch.optim as optim

t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

def model(t_u, w, b):
    return w*t_u + b

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#随机获取训练集与验证集索引
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

#train_indices, val_indices
#(tensor([ 0,  5,  8,  1, 10,  7,  9,  4,  6]), tensor([3, 2])) #结果随机

#根据索引划分训练集与验证集
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

#将t_u进行规范化
train_t_un = 0.1 * train_t_u
val_t_un = 0.1 *val_t_u

def training_loop(n_epochs, optimizer, params,
                  train_t_u, val_t_u, train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        #不需要时关闭autograd
        with torch.no_grad():
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        if epoch <= 3 or epoch % 500 == 0:
            print('Epoch %d, Traing loss % .2f, Validation loss % .2f'%(
                    epoch, float(train_loss), float(val_loss)))
    return params

params = torch.tensor([1.0, 0.0], requires_grad = True) #初始化参数并激活梯度
learning_rate = 1e-2 #设置学习率
optimizer = optim.SGD([params], lr = learning_rate) #设定优化器

training_loop(
    n_epochs = 3000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un,
    val_t_u = val_t_un,
    train_t_c = train_t_c,
    val_t_c = val_t_c)


Epoch 1, Traing loss  75.80, Validation loss  100.93
Epoch 2, Traing loss  38.23, Validation loss  44.65
Epoch 3, Traing loss  31.55, Validation loss  32.36
Epoch 500, Traing loss  6.49, Validation loss  11.69
Epoch 1000, Traing loss  3.02, Validation loss  7.49
Epoch 1500, Traing loss  2.52, Validation loss  6.17
Epoch 2000, Traing loss  2.45, Validation loss  5.71
Epoch 2500, Traing loss  2.44, Validation loss  5.55
Epoch 3000, Traing loss  2.44, Validation loss  5.48


tensor([  5.2245, -16.4587], requires_grad=True)